In [1]:
%%capture
!pip install rank-bm25
!pip install stopwordsiso

In [2]:
import stopwordsiso
import pandas as pd
from rank_bm25 import BM25Okapi
import pandas as pd
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import nltk

In [3]:
nltk.download('punkt')
nltk.download('stopwords')
stop_words = stopwordsiso.stopwords("hr")
print(len(stop_words))

[nltk_data] Downloading package punkt to /usr/share/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
179


In [4]:
dataset_path = "/kaggle/input/wiki-and-ms-marco"
test_df = pd.read_csv(dataset_path + "/test.csv")

In [5]:
def preprocess(text):
    """Tokenize and remove stopwords."""
    tokens = word_tokenize(text.lower())
    return [word for word in tokens if word.isalnum() and word not in stop_words]

test_df['processed_passage'] = test_df['passage'].apply(preprocess)
corpus = test_df['processed_passage'].tolist()

bm25 = BM25Okapi(corpus)

In [6]:
results = []
for i, row in test_df.iterrows():
    query = row['query']
    relevant_passage = row['passage']
    query_tokens = preprocess(query)

    scores = bm25.get_scores(query_tokens)

    sorted_indices = scores.argsort()[::-1]
    sorted_scores = scores[sorted_indices]
    sorted_passages = [test_df.iloc[idx]['passage'] for idx in sorted_indices]

    relevant_idx = sorted_indices.tolist().index(i)
    relevant_score = scores[i]
    rank = relevant_idx + 1

    results.append({
        'query': query,
        'retrieved_passage': sorted_passages[0],
        'retrieved_score': sorted_scores[0],
        'relevant_passage': relevant_passage,
        'relevant_score': relevant_score,
        'rank': rank
    })

results_df = pd.DataFrame(results)

results_df.to_csv("retrieved_results_with_rank.csv", index=False)


In [7]:
def mean_reciprocal_rank(ranks):
    return sum(map(lambda x: 1/x, ranks)) / len(ranks)

def hit_rate_at_1(ranks):
    return sum(map(lambda x: x==1, ranks)) / len(ranks)

In [8]:
ranks = list(results_df["rank"])
mrr = mean_reciprocal_rank(ranks)
hr = hit_rate_at_1(ranks)
print(f"MRR: {mrr}, HR@1: {hr}")

MRR: 0.7984746956270221, HR@1: 0.7501259445843829


In [9]:
results_df

,query,retrieved_passage,retrieved_score,relevant_passage,relevant_score,rank
0,koliko je star dr.nowzaradan,Koliko dugo nakon ovulacije jaje ostaje održiv...,7.072334,Dr. Nowzaradan Plan prehrane.1 muškarci.2 treb...,0.000000,1985
1,Kako ukloniti anketu,Da biste izbrisali anketu: 1 Idite na stranicu...,13.279188,Da biste izbrisali anketu: 1 Idite na stranicu...,13.279188,1
2,Kako bih mogao razumjeti Dolića Draga?,"Dolića Draga je naselje u općini Lokvičići, u ...",23.324747,"Dolića Draga je naselje u općini Lokvičići, u ...",23.324747,1
3,Što MFD znači za pisač,Značenje kanonskog pisača za upozorenje svjetl...,20.350662,Značenje kanonskog pisača za upozorenje svjetl...,20.350662,1
4,Koji su organi uključeni u sustav izlučivanja,Organi i njihove funkcije.1 Bubrezi otpadu iz ...,9.293178,Organi i njihove funkcije.1 Bubrezi otpadu iz ...,9.293178,1
...,...,...,...,...,...,...
1980,vrijedi li ipsy torba,Pozdrav moji ljubavnici!Pretplaćen sam na Ipsy...,18.223156,Pozdrav moji ljubavnici!Pretplaćen sam na Ipsy...,18.223156,1
1981,Definicija CRM mogućnosti,Voditelji su potencijalni ili potencijalni kup...,19.879169,Voditelji su potencijalni ili potencijalni kup...,19.879169,1
1982,Što je Hallux,Prosječna plaća financijskih analitičara iznos...,0.000000,Bunions su progresivni poremećaj.Započinju nag...,0.000000,1333
1983,Kratak uvod u NGC 1163.,NGC 1163 je galaksija u zviježđu Eridan.,18.757347,NGC 1163 je galaksija u zviježđu Eridan.,18.757347,1
